In [1]:
import deeplabcut 
import os
from glob import glob
from tqdm import trange, tqdm
from tkinter import filedialog
import tkinter as tk

C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type

In [2]:
config_path = r'C:\Users\Niell Lab\Documents\deeplabcut_projects\EphysEyeCams3-dylan-2021-02-02\config.yaml'

In [ ]:
new_vids = glob(r'C:/Users/Niell Lab/Desktop/EphysEyeCams3_retrain_videos/*.avi')
new_vids

In [ ]:
len(new_vids)

In [ ]:
deeplabcut.add_new_videos(config_path,new_vids,copy_videos=True)

In [ ]:
deeplabcut.extract_frames(config_path, mode='automatic', algo='uniform', userfeedback=False, crop=False)

In [ ]:
deeplabcut.label_frames(config_path)

In [3]:
deeplabcut.merge_datasets(config_path)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


In [4]:
deeplabcut.create_training_dataset(config_path, augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([308, 116, 339, 265,  63, 200, 144,  26, 170, 249, 189, 355, 259,
          103, 218, 331,  96, 135, 241, 117, 157,  51, 276, 392, 102,  71,
          325,  32, 404, 382, 319, 210, 224,  55, 244, 231, 175, 164,  69,
          380, 287, 159, 293, 233,  54,  13, 150, 328, 385, 290, 148, 303,
          173,  28,  70, 248, 269, 408, 152,  38, 371, 346, 208, 388,  23,
          262, 216, 106,  30, 132, 107, 176, 171, 294, 167,  22, 309, 145,
          221, 271,  15, 361, 381,  83, 203, 376, 406, 356, 379, 125,  88,
          124, 395,  20, 264, 250, 182, 377,   5, 318, 285, 198, 215,  29,
          357,   3, 344, 364,  90,  68, 310, 297, 178, 370, 147, 272, 246,
           82,  85, 394,  52, 295, 104,  34,  81, 365,  39,  41, 146, 245,
          206,  58, 373,  84,   4, 281, 192, 267,  46,  57,  42, 402, 252,
          396, 257, 213, 191, 205,  33, 315, 279, 300, 327, 236,  87,  12,
          283,  40, 347,  10, 123, 314, 156, 277, 247, 253, 242, 121,  86,
          16

In [5]:
# Set up pose_config.yaml
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)
# Add imageaug to pose_config.yaml 
trainposeconfigfile,testposeconfigfile,snapshotfolder=deeplabcut.return_train_network_path(config_path)
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)
cfg_dlc['scale_jitter_lo']= 0.5
cfg_dlc['scale_jitter_up']=1.5
cfg_dlc['augmentationprobability']=.5
cfg_dlc['batch_size']=6 #pick that as large as your GPU can handle it
cfg_dlc['elastic_transform']=True
cfg_dlc['rotation']=180
cfg_dlc['covering']=True
cfg_dlc['motion_blur'] = True
cfg_dlc['optimizer'] ="adam"
cfg_dlc['dataset_type']='imgaug' # 'imagaug'multi-animal-
cfg_dlc['multi_step']=[[1e-4, 7500], [5.0e-5, 12000], [1e-5, 50000]]
cfg_dlc['init_weights'] = 'C:/Users/Niell Lab/Documents/deeplabcut_projects/EphysEyeCams3-dylan-2021-02-02/dlc-models/iteration-0/EphysEyeCams3Feb2-trainset95shuffle1/train/snapshot-800000'
deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

In [6]:
# Train and Evaluate Network
deeplabcut.train_network(config_path, allow_growth=True, displayiters=500, maxiters=100000, saveiters=10000)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['pt1',
                      'pt2',
                      'pt3',
                      'pt4',
                      'pt5',
                      'pt6',
                      'pt7',
                      'pt8',
                      'tear',
                      'outer'],
 'augmentationprobability': 0.5,
 'batch_size': 6,
 'covering': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_EphysEyeCams3Feb2\\EphysEyeCams3_dylan95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'display_iters': 1000,
 'elastic_transform': True,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:/Users/Niell '
                 'Lab/Documents/deeplabcut_projects/EphysEyeCams3-dylan-2021-02-02/dlc-models/iteration-0/EphysEyeCams3Feb2-trainset95shuffle1/train/snapshot-800000',
 'intermediate_supervision': False,
 'intermediate_superv

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 6
Initializing ResNet
Loading already trained DLC with backbone: resnet_50
Max_iters overwritten as 50000
Display_iters overwritten as 500
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\Niell Lab\\Documents\\deeplabcut_projects\\EphysEyeCams3-dylan-2021-02-02\\dlc-models\\iteration-1\\EphysEyeCams3Feb2-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'adam', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 6, 'dataset_type': 'imgaug', 

iteration: 500 loss: 0.0078 lr: 0.0001
iteration: 1000 loss: 0.0065 lr: 0.0001
iteration: 1500 loss: 0.0060 lr: 0.0001
iteration: 2000 loss: 0.0059 lr: 0.0001
iteration: 2500 loss: 0.0057 lr: 0.0001
iteration: 3000 loss: 0.0054 lr: 0.0001
iteration: 3500 loss: 0.0055 lr: 0.0001
iteration: 4000 loss: 0.0054 lr: 0.0001
iteration: 4500 loss: 0.0051 lr: 0.0001
iteration: 5000 loss: 0.0049 lr: 0.0001
iteration: 5500 loss: 0.0052 lr: 0.0001
iteration: 6000 loss: 0.0050 lr: 0.0001
iteration: 6500 loss: 0.0050 lr: 0.0001
iteration: 7000 loss: 0.0048 lr: 0.0001
iteration: 7500 loss: 0.0050 lr: 0.0001
iteration: 8000 loss: 0.0043 lr: 5e-05
iteration: 8500 loss: 0.0040 lr: 5e-05
iteration: 9000 loss: 0.0041 lr: 5e-05
iteration: 9500 loss: 0.0040 lr: 5e-05
iteration: 10000 loss: 0.0041 lr: 5e-05
iteration: 10500 loss: 0.0041 lr: 5e-05
iteration: 11000 loss: 0.0039 lr: 5e-05
iteration: 11500 loss: 0.0041 lr: 5e-05
iteration: 12000 loss: 0.0039 lr: 5e-05
iteration: 12500 loss: 0.0035 lr: 1e-05
itera

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\train.py", line 91, in load_and_enqueue
    sess.run(enqueue_op, feed_dict=food)
  File "C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\tensorflow\python\client\session.py", line 929, in run
    run_metadata_ptr)
  File "C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\tensorflow\python\client\session.py", line 1075, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



In [7]:
deeplabcut.evaluate_network(config_path, plotting=True, trainingsetindex='all')

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['pt1',
                      'pt2',
                      'pt3',
                      'pt4',
                      'pt5',
                      'pt6',
                      'pt7',
                      'pt8',
                      'tear',
                      'outer'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_EphysEyeCams3Feb2\\EphysEyeCams3_dylan95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\Niell '
                 'Lab\\.conda\\envs\\DLC-GPU2\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 

C:\Users\Niell Lab\Documents\deeplabcut_projects\EphysEyeCams3-dylan-2021-02-02/evaluation-results/  already exists!
Running  DLC_resnet50_EphysEyeCams3Feb2shuffle1_50000  with # of trainingiterations: 50000
Initializing ResNet


0it [00:00, ?it/s]

Analyzing data...


409it [00:11, 35.57it/s]
  0%|                                                                                          | 0/409 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-50000
Results for 50000  training iterations: 95 1 train error: 2.27 pixels. Test error: 3.09  pixels.
With pcutoff of 0.6  train error: 2.27 pixels. Test error: 3.09 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [01:19<00:00,  5.12it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)
